Check python version 

change python version through menu: Runtime -> Change runtime type

choose python 3 and gpu

In [ ]:
# change python version through menu: Runtime -> Change runtime type
# choose python 3 and gpu
import sys
print(sys.version)

Mount drive 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Install libraries

In [ ]:
%pip install -U PyYAML
%pip install neat-python torch
%pip install chainer
# might become necessary later
# pip install 'cupy-cuda111>=7.7.0,<8.0.0'

Pull code from github

In [ ]:
 !rm -r illusion_evolver
 !git clone https://github.com/LanaSina/illusion_evolver.git --recurse-submodules

Download prediction models: https://figshare.com/articles/dataset/Illusions_Models/19874077

Then drag and drop to the left of this screen and skip to the next celle, or save in your google drive and change the path below to your own drive.

In [ ]:
# color
!cp '/content/drive/My Drive/AI/prednet_private/models/fpsi_500000_20v.model' '.'

# black and white
# !cp '/content/drive/My Drive/AI/prednet_private/models/black_and_white/300000_wb.model' '.'

Generate illusions. The results will be saved to the "results" folder on the left. The first illusions will be bad, and should improve with time. Stop the algorithm whenever you are satistified.


In [ ]:
!rm -r "results"
!rm -r "temp"

# argument '-s': Type of illusion. 0: Bands; 1: Circles (default=0)
# %run 'illusion_evolver/generate_illusion.py' -m 'fpsi_500000_20v.model' -s 1 -o './results'

# black and white
%run 'illusion_evolver/generate_illusion.py' -m '300000_wb.model' -s 1 -c 1 -o './results' --channels "1,16,32,64" -g -1

Dowload results by left-click and "save image as", or download whole folder.



In [ ]:
# download images
!zip -r "results.zip" "results" 
from google.colab import files
files.download('results.zip') 

If you want to get the visual flow.

In [ ]:
%run "illusion_evolver/optical_flow/optical_flow.py" "debug/0000000004.png" "debug/0000000005_extended.png" -s 0 -l 1 -cc yellow -lc red -s 2 -l 2 -vs 60.0